# Семинар 5 🏯 Архитектуры сверточных нейронных сетей 🏯

Рассмотрим основные архитектуры сверточных нейронных сетей, с чего они начинались и как они эволюционировали.
Эти модели могут использоваться по-разному: для классификации или для извлечения признаков (детекция, сегментация и т.д.)

# LeNet (Yann LeCun (Bell Labs, now Meta) et al. in 1989)

- Первое успешное применение обратного распространения на практике.
Задача - распознавание рукописных цифр zip-кода для почты США (accuracy на `MNIST` составляет около 98%)

- На `Cifar-10` около 60%

- Архитектура: 2 x conv (kernel=5x5), 2 x padding(stride=2), 3 x FC. Активация - сигмоида

- 60k trainable parameters

[О LeNet на medium.com](https://medium.com/mlearning-ai/lenet-and-mnist-handwritten-digit-classification-354f5646c590)

![](images/0.png)

In [ ]:
import torch
from torch import nn

le_net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), 
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), 
    nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), 
    nn.Sigmoid(),
    nn.Linear(120, 84), 
    nn.Sigmoid(),
    nn.Linear(84, 10),
)

# Датасет ImageNet

- Классификация изображений, задача [`Imagenet`](https://www.image-net.org/) (1000 классов, 1М изображений). Примеры классов: животные (например, собаки разной породы), техника, автомобили, разные предметы

- Human-level performance на этом датасете: top-5 error rate: 5.1%

- SOTA на этом датасете: top-5 error rate: ~ 1% (то есть нейросети уже превзошли человека человека)

- Начиная с `AlexNet`, ошибка нейронок сильно уменьшилась

![](images/1.png)

![](images/2.png)

# [AlexNet](https://proceedings.neurips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf) (Alex Krizhevsky, Ilya Sutskever and Geoffrey Hinton, 2012)

![](images/3.png)

- Как `Lenet`, но больше
- ~ 15% top-5 error rate на Imagenet
- 60 M параметров
- Активация: ReLU
- Обучалась на нескольких GPU (на тот момент не влезала на одну GPU, учили на двух)
- Пулинг с оверлапом (просто потестили и заметили, что метрика так получается выше)
- Dropout, чтобы не переобучаться

[Об AlexNet](https://neurohive.io/ru/vidy-nejrosetej/alexnet-svjortochnaja-nejronnaja-set-dlja-raspoznavanija-izobrazhenij/)

In [ ]:
alex_net = nn.Sequential(
    # Here, we use a larger 11 x 11 window to capture objects. At the same
    # time, we use a stride of 4 to greatly reduce the height and width of the
    # output. Here, the number of output channels is much larger than that in
    # LeNet
    nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # Make the convolution window smaller, set padding to 2 for consistent
    # height and width across the input and output, and increase the number of
    # output channels
    nn.Conv2d(96, 256, kernel_size=5, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # Use three successive convolutional layers and a smaller convolution
    # window. Except for the final convolutional layer, the number of output
    # channels is further increased. Pooling layers are not used to reduce the
    # height and width of input after the first two convolutional layers
    nn.Conv2d(256, 384, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten(),
    # Here, the number of outputs of the fully-connected layer is several
    # times larger than that in LeNet. Use the dropout layer to mitigate
    # overfitting
    nn.Linear(6400, 4096),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    # Output layer. Since we are using Fashion-MNIST, the number of classes is
    # 10, instead of 1000 as in the paper
    nn.Linear(4096, 10),
)

# [VGG](https://arxiv.org/abs/1409.1556) (Karen Simonyan, Andrew Zisserman, 2014)

- Улучшенная версия `AlexNet` - свертки 11x11 и 5x5 заменены на 3x3
- Исследовали влияние глубины сети на метрику на `ImageNet`, подобрали 2 архитектуры: `VGG-16` и `VGG-19`
- conv3x3 → conv3x3 → maxpool → conv3x3 → conv3x3 → maxpool → ... → FC
- 6.8% top-5 error rate
- ~ 140M параметров
- тяжелая (VGG-16 весит около 500 Мб, а ResNet-18 45 Мб)
- не получается сделать очень глубокой

[О VGG на towards data science](https://towardsdatascience.com/vgg-neural-networks-the-next-step-after-alexnet-3f91fa9ffe2c)

*Упражнение: покажите, что рецептивное поле свёрток 5x5 и двух последовательных свёрток 3x3 равны. Как при этом различается число операций?*

![](images/4.png)

![](images/5.png)

Чтобы поменять число классов для своей задачи, переопределите последний FC-слой у модели:

In [ ]:
from torchvision.models import vgg16
model = vgg16()
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
model.classifier[6] = nn.Linear(in_features=4096, out_features=42)

# [GoogLeNet (Inception V1, 2014)](https://arxiv.org/abs/1409.4842)

![](images/we_need_to_go_deeper.jpg)

`GoogLeNet` выиграла `ImageNet Recognition Challenge` в 2014 с top5 error rate=6,67% (для сравнения, у `AlexNet` 15% top5 errror rate). У `AlexNet` 60M параметров, и исследователи из Google решили придумать, как сделать её поменьше и попрактичнее. Вот как они рассуждали:

- Самый простой способ сделать сеть точнее - это увеличить её размер (то есть ширину и глубину)
- К сожалению, с увеличением размера сети она становится более подверженной переобучению (попросту потому, что у большой сети больше параметров) и у нее растет вычислительная сложность
- Есть еще проблема разных масштабов: предположим, что мы учим нейросеть для классификации животных. Область картинки, по которой мы принимаем решение о классе, может быть очень разного масштаба (см. рисунок ниже). Из-за этого непонятно, какой kernel size брать, побольше или поменьше (побольше - когда собачка занимает почти весь кадр, поменьше - когда очень маленькую часть)
- А давайте не будем выбирать kernel size - возьмем просто много разных для разных масштабов. Так мы сможем работать с разным масштабом фичей.
- Чтобы вычислительная сложность не страдала, будем пользоваться трюками - например, перед свертками $5\times 5$ вставлять свертку $1\times 1$ для изменения числа каналов.
- Перед inception-блоком вставляем свертку $1\times 1$ для уменьшения числа каналов
- Для уменьшения изображения inception-блоки чередуются с `maxpool` со страйдом 2
- Сделаем параллельные ветки для фичей разного размера → наша сеть станет скорее не глубокой, а широкой
- Не делаем скрытого fc-слоя вообще. Берем global average pooling, а после него сразу ставим финальный fc-слой

[О разных версиях Inception на towards data science](https://towardsdatascience.com/a-simple-guide-to-the-versions-of-the-inception-network-7fc52b863202)

![](images/6.png)

![](images/7.png)

*Inception module, наивная версия. В разных ветках используется паддинг same*

![](images/8.png)

*Здесь уже вставлены свертки $1\times 1$ для уменьшения количества каналов*

In [ ]:
class Inception(nn.Module):
    # `c1`--`c4` are the number of output channels for each path
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # Path 1 is a single 1 x 1 convolutional layer
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # Path 2 is a 1 x 1 convolutional layer followed by a 3 x 3
        # convolutional layer
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # Path 3 is a 1 x 1 convolutional layer followed by a 5 x 5
        # convolutional layer
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # Path 4 is a 3 x 3 maximum pooling layer followed by a 1 x 1
        # convolutional layer
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # Concatenate the outputs on the channel dimension
        return torch.cat((p1, p2, p3, p4), dim=1)

![](images/9.png)

*GoogLeNet (InceptionV1), архитектура сети. Обведены в прямоугольник два дополнительных классификатора (сделаны для борьбы с затухающими градиентами)*

# [InceptionV2](https://arxiv.org/pdf/1512.00567v3.pdf)

Авторы предложили ряд улучшений для архитектуры `InceptionV1`:

- Факторизуем свертки $5\times5$ на две свертки $3\times3$
- Факторизуем свертки $n\times n$ на две свертки $n× 1$ и $1\times n$

![](images/10.png)

*Доп. вопрос: как такие факторизации сверток изменят количество вычислительных операций?*

# [ResNet](https://arxiv.org/abs/1512.03385) (Kaiming He, Xiangyu Zhang, 2015)

После `LeNet` появлялись все новые и новые архитектуры, сети становились глубже и показывали большие точности. Возникал вопрос: а что если просто сделать нейросеть очень глубокой, наверное, станет ещё лучше?

Оказывается, что это не так: с добавлением слоёв точность сначала насыщается (то есть перестаёт меняться), а потом начинает ухудшаться. И это не проблема переобучения, потому что на тренировочном датасете точность тоже становится хуже:

![](images/11.png)

*Training error сетей с 20 и 56 слоями на датасете CIFAR-10*

В чём идея `ResNet`: 
- Пусть у нас есть сеть, которая хорошо выучилась на нашем датасете. Мы можем добавить к архитектуре предыдущей сети несколько слоёв и попытаться её обучать. По-хорошему, наша новая архитектура должна быть не хуже предыдущей, потому что сеть может при помощи дополнительных слоёв, которые мы добавили, просто выучить тождественное преобразование.
- На практике такая сеть будет иметь худшую точность
- Попробуем помочь сети: блок сети будет теперь учить добавку к исходным данным $y = x + F(x)$ (блок со skip connection)
- Когда сеть еще не натренированная, то градиенту проще протечь на предыдущие слои за счёт единички: $E^\prime_x = E^\prime_y (1+F^\prime_x)$

![](images/12.png)

*Доп. вопросы:*

*- Какие веса/сдвиги должна выучить сеть, чтобы получить тождественное преобразование $y = x + F(x)$?*

*- Как течет градиент, если у нас есть несколько веток или skip connection?*


### ResNet-блоки

Какие у нас виды блоков есть в сетях `ResNet`:

- **Identity block** - стандартный блок, когда размер входа совпадает с размером выхода (в skip connection просто $x$, identity shortcut)
- **Convolutional block** - блок, в котором размер входа и размер выхода не совпадают (в skip connection свертка $x$, projection shortcut) 

И identity, и convolutional блоки могут иметь две или три свертки в $F(x)$. Когда их две, то такой блок **residual block** называется **классическим  residual block'ом**, а когда три - **bottlneck residual block**. Bottleneck используется в `ResNet-50` и еще более глубоких вариантах архитектуры

**Identity block**

- Либо две (классический блок), либо три (bottleneck блок) свертки в ветке с $F(x)$
- $F(x)$ не меняет размер тензора, входящего в блок, поэтому можно просто сложить $F(x) + x$

![](images/13.png)

**Convolutional block**

- либо две, либо три свёртки в нижней ветке $F(x)$ (зависит от того, классический или bottleneck block), внутри блока есть свертка со страйдом 2
- свёртка в верхней ветке: $Conv_{1\times1}$ со страйдом 2, чтобы можно было сложить с $F(x)$

![](images/14.png)

### Сравнение классического и bottleneck блока

**Классический блок:**
- В классическом блоке 2 свертки в ветке $F(x)$, обе $3\times3$, $c_{in}$ = $c_{out}$, паддинг same (дополняющий до исходного размера)
- Если надо уменьшить пространственный размер (convolutional block), то первая свертка $3\times3$ со страйдом 2

**Bottleneck блок:**

- На входе у нас $c$ каналов, внутри блока каналов становится меньше (первые две свертки), а затем число каналов снова увеличивается - получается "бутылочное горлышко". Это позволяет увеличить глубину сети без резкого увеличения количества параметров. Сжимаем большое число каналов при помощи "дешевых" $1\times1$ сверток, а на выходе из блока увеличиваем число каналов.

- Bottleneck identity блок:
  - $Conv_{1\times1}$
  - $Conv_{3\times3}$
  - $Conv_{1\times1}$
- Bottleneck convolutional блок:
  - $Conv_{1\times1}$ ← можно страйд 2 сюда, тогда вычислений меньше
  - $Conv_{3\times3}$ ← а можно сюда, тогда будет точнее
  - $Conv_{1\times1}$

![](images/15.png)

Подробнее как выглядят разные варианты архитектуры:

![](images/16.png)

*Блоки модели указаны в квадратных скобках, рядом после $\times$ - число повторений. $conv3\_1, conv4\_1, conv5\_1$ осуществляют downsampling*

С появлением `ResNet` произошла революция, сети стали гораздо глубже:

![](images/17.png)

При этом в `VGG-16` 138M параметров, а у `ResNet34` 22M, легче модель и проще обучать. А еще эта модель работает с любым размером изображения из-за adaptive average pooling в конце.

Благодаря всем этим плюсам, `ResNet` в 2015 году добился top-5 error rate в 3.6% на `ImageNet` и победил в соревновании

[Видео на русском про ResNet](https://www.youtube.com/watch?v=QlC3Qt2Jl0M), [Статья на хабр](https://habr.com/ru/company/vk/blog/311706/)

В чем разница `ResNetv1` и `ResNetv2`?

Предложили ставить `ReLU` не после суммы, а до, чтобы градиент лучше тёк через skip connections:

![](images/18.png)

# [MobileNet (2017)](https://arxiv.org/pdf/1704.04861.pdf)

В наши дни никого не удивить тем, что нейросети используют не только вместе со специальным железом. Примеры задач, для которых размер модели (Мб) и скорость работы являются ключевыми факторами:

- Распознавание лиц ("умный домофон")
- Детектирование и распознавание автомобильных номеров (умная камера DAHUA)
- Разблокировка телефона по лицу
- Self-driving cars

Для того, чтобы нейросеть смогла работать на edge устройствах, она должна быть легкой и быстрой. Как раз такой архитектурой и является `MobileNet`.
Коротко об этой сети:
- Используются **depthwise** и **pointwise** ($1\times 1$) свёртки
- У архитектуры есть два гиперпараметра, подстраивая которые можно выбирать оптимальный вариант между производительностью и точностью модели. Они называются **width multiplier** и **resolution multiplier**

[Статья на хабр про MobileNet](https://habr.com/ru/post/352804/)

![](images/19.png)

*Слева: Стандартный сверточный слой с батчнормом и активацией. Справа: вариант с факторизацией на depthwise и pointwise*

$FLOPS_1 / FLOPS_2 = (9\times C_{in}\times hw + C_{in}\times C_{out}\times hw) / (9\times C_{in}\times C_{out}\times hw) = 1 / C_{out} + 1 / 9 \ll 1$

Все свёртки $3\times 3$ (кроме самой первой) факторизованы на depthwise и pointwise. Между свертками есть `BatchNorm` и `ReLU`, всего 28 слоёв

![](images/20.png)

![](images/21.png)

Большая часть вычислений приходится на pointwise свёртки

**Width multiplier $α$**

Этот параметр нужен для того, чтобы иметь возможность равномерно сузить сеть на каждом слое: $C_{in} → α\times C_{in}$, $C_{out} → α\times C_{out}$. Единице соответсвует стандартный `MobileNet`

**Resolution multiplier $\rho$**

Домножаем входное разрешение на этот коэффициент $\rho$. Стандартный вход $224\times224$, с домножением могут получиться $192, 160, 128...$

*Вопрос: как меняется сложность сети с измененением $\alpha$ и $\rho$?*

- Факторизация сверток $3\times 3$ на depthwise и pointwise свертки в `MobileNet` привела к уменьшению top-1 accuracy на всего 1%, а число операций уменьшилось почти в 1000 раз
- При помощи $\alpha$ и $\rho$ можно регулировать вес модели и скорость, но теряется точность
- top-5 accuracy ~ 90%
- Точность чуть ниже, чем у `VGG-16`, а число параметров сильно меньше 138.35M → 4.20M

![](images/22.png)


 # [MobileNet V2 (2019)](https://arxiv.org/pdf/1801.04381.pdf)

**Inverted Residuals**

- В `MobileNetV2` появился residual block. В оригинальном residual блоке в `ResNet` число каналов сужалось и затем увеличивалось обратно ("бутылочное горлышко"). Свёртки $1\times1$ уменьшали число каналов на входе в блок, за счет чего свертка $3\times3$ становилась "дешевле"
- В `MobileNetv2` решили использовать другой подход - внутри блока каналы увеличиваются, а на выходе сужаются. У нас получился "residual блок наборот", отсюда и название "inverted residuals"
- У нас как бы эффективно есть внутреннее представление данных - "целевое многообразие" (manifold of interest) большей размерности. Мы проводим вычисления (свертку $3\times 3$) в этом пространстве большей размерности, а затем снова "укладываем" данные в подпространство меньшей размерности. Чтобы число вычислений не становилось огромным, используется depthwise свёртка $3\times 3$
- В inverted residual блоке меньше параметров, чем в обычном residual, что хорошо, т.к. мы хотим более быструю сеть с меньшим числом параметров
- Мы можем увеличивать число каналов внутри блока в $t$ раз (авторы рассматривают $t\in[5,10]$, где меньшие значения лучше работают для меньших сетей, а большие - для больших. В классическом варианте $t=6$.
- Если нужно понизить размерность, то страйд 2 добавляется к $3\times 3$ depthwise свёртке.


**Linear Bottleneck**

- Мы используем в качестве функции активации `ReLU6`, но она зануляет часть диапазона и поэтому ведет к потере части информации
- Авторы решили, что при большом числе каналов этот эффект должен компенсироваться
- На выходе первой и второй свертки есть активация `ReLU6`, а на выходе последней свертки в блоке (как раз там, где мы сжимаем число каналов) её нет

![](images/23.png)

## MobileNetV3

`MobileNetV3` отличается от предыдущих версий тем, что в нём появился т.н. [Squeeze-and-Excitation block](https://towardsdatascience.com/squeeze-and-excitation-networks-9ef5e71eacd7)